In [1]:
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
import pandas as pd
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, classification_report, precision_recall_curve
import numpy as np
import plotly.express as px
# import shap
import seaborn as sns
from IPython import display
import matplotlib.pyplot as plt
import awswrangler as wr
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    classification_report,
    multilabel_confusion_matrix,
)
from catboost.utils import select_threshold
from skmultilearn.model_selection.iterative_stratification import (
    IterativeStratification,
)
from sklearn.model_selection import StratifiedKFold, KFold
from tqdm import tqdm
from catboost.utils import get_roc_curve, select_threshold
from torchmetrics.classification import BinaryPrecisionRecallCurve
# from supervised.automl import AutoML
import warnings
from loguru import logger
from sklearn.metrics import f1_score, precision_score, recall_score
from catboost.utils import eval_metric
from catboost import metrics as cat_metrics
from sklearn.utils import class_weight

warnings.filterwarnings("ignore")


/home/alextay96/.local/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/alextay96/.local/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/alextay96/.local/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator9allocatorE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
wr.config.s3_endpoint_url = "http://192.168.1.4:8333"

multilabelDf = wr.s3.read_parquet(
    path=f"s3://multilabel_df/",
    dataset=True,
)
multilabelDf.columns

Index(['vision_bonnet', 'vision_bumper_front', 'vision_grille',
       'vision_headlamp_rh', 'vision_headlamp_lh', 'vision_door_front_lh',
       'vision_door_front_rh', 'vision_engine', 'vision_bumper_rear',
       'vision_misc', 'vision_front_panel', 'vision_non_external',
       'vision_wheel', 'vision_fender_front_lh', 'vision_fender_front_rh',
       'vision_rear_quarter_lh', 'vision_tail_lamp_lh', 'vision_tail_lamp_rh',
       'vision_windscreen_front', 'vision_rear_compartment',
       'vision_rear_panel', 'vision_rear_quarter_rh', 'vision_door_rear_rh',
       'vision_door_rear_lh', 'bonnet', 'bumper_front', 'grille',
       'fog_lamp_rh', 'headlamp_lh', 'headlamp_rh', 'door_front_lh',
       'door_front_rh', 'air_conditioning', 'cooling_fan', 'radiator',
       'bumper_rear', 'misc', 'engine', 'airbag', 'front_panel',
       'undercarriage', 'wheel', 'fender_front_lh', 'fender_front_rh',
       'door_mirror_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh',
       'fog_lamp_l

In [3]:
# multilabelDf.replace(pd.NA, 0, inplace=True)

In [4]:
multilabelDf["Assembly_Type"]

0         3
1         3
2         3
3         3
4         3
         ..
598121    3
598122    3
598123    3
598124    2
598125    3
Name: Assembly_Type, Length: 598126, dtype: Int64

In [5]:
multilabelDf[multilabelDf["Model"] == "NA"]

,vision_bonnet,vision_bumper_front,vision_grille,vision_headlamp_rh,vision_headlamp_lh,vision_door_front_lh,vision_door_front_rh,vision_engine,vision_bumper_rear,vision_misc,...,Assembly_Type,Vehicle_Still_Driveable,NCB_Stat,Claim_Type,Vehicle_Type,Sum_Insured,Repairer,Repairer_Apprv_Count,Collision_With,Handling_Insurer


In [6]:
wr.config.s3_endpoint_url = "http://192.168.1.4:8333"

multilabelDf = wr.s3.read_parquet(
        path=f"s3://multilabel_df/",
        dataset=True,
    )
allVisionFeatures = [x for x in multilabelDf.columns if "vision_" in x]
caseFeatures = [
    "Circumstances_of_Accident",
    "Model",
    # "Vehicle_Still_Driveable",
    # "NCB_Stat",
    # "Assembly_Type",
    # "Claim_Type",
    "Vehicle_Type",
    # # "Sum_Insured",
    # "Repairer",
    # # # "Repairer_Apprv_Count",
    # "Collision_With",
    # "Handling_Insurer",
]
exclude = ['Sum_Insured', 'Repairer', 'Repairer_Apprv_Count', 
"Vehicle_Still_Driveable",
    "NCB_Stat",
    "Assembly_Type",
    "Claim_Type",
    # "Vehicle_Type",
    "Sum_Insured",
    "Repairer",
    "Repairer_Apprv_Count",
    "Collision_With",
    "Handling_Insurer"
     ]
# contFeatures = ["Sum_Insured", "Repairer_Apprv_Count"]
allInputFeature = caseFeatures + allVisionFeatures
allTargetCol = [
   'bonnet', 'bumper_front', 'grille', 'fog_lamp_rh', 'headlamp_lh', 'headlamp_rh', 'door_front_lh', 'door_front_rh', 'air_conditioning', 'cooling_fan', 'radiator', 'bumper_rear', 'misc', 'engine', 'airbag', 'front_panel', 'undercarriage', 'wheel', 'fender_front_lh', 'fender_front_rh', 'door_mirror_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh', 'fog_lamp_lh', 'tail_lamp_rh', 'windscreen_front', 'rear_compartment', 'rear_panel', 'rear_quarter_rh', 'door_rear_rh', 'door_mirror_lh', 'door_rear_lh', 'windscreen_rear'
]
saloonPredDf = pd.read_csv(
    "/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/saloon_new_labels_pos_weight/Saloon - 4 Dr_imgs_pred_output.csv"
)
hatchBackDf = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/hatchback_new_labels_pos_weight/Hatchback - 5 Dr_imgs_pred_output.csv")
suvDf = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/results/suv_new_labels_pos_weight/SUV - 5 Dr_imgs_pred_output.csv")

imgPredOutput = pd.concat([saloonPredDf, hatchBackDf, suvDf])
# imgPredOutput = saloonPredDf
allPredModel = []
allPreds = []
allGt = []
realTestDataDf = multilabelDf[caseFeatures + allTargetCol + ["CaseID"]].merge(
    imgPredOutput, on="CaseID"
)
realTestDataDf = realTestDataDf.loc[
    :, ~realTestDataDf.columns.str.contains("^Unnamed")
]
trainDf = multilabelDf[
    ~multilabelDf["CaseID"].isin(realTestDataDf["CaseID"].unique().tolist())
]
# trainDf = trainDf.sample(frac=0.5)
assert set(realTestDataDf["CaseID"].tolist()).isdisjoint(trainDf["CaseID"].tolist())
targetPart = [
    "fog_lamp_rh",
    "airbag",
    "undercarriage",
    "door_mirror_rh",
    "interior",
    "fog_lamp_lh",
    "door_mirror_lh",
    "windscreen_rear",
    "air_conditioning",
    "cooling_fan",
    "radiator"
]
# targetPart = [
#      'bonnet', 'bumper_front', 'grille', 'headlamp_lh', 'headlamp_rh', 'door_front_lh', 'door_front_rh', 'bumper_rear', 'misc', 'engine', 'front_panel', 'wheel', 'fender_front_lh', 'fender_front_rh', 'rear_quarter_lh', 'interior', 'tail_lamp_lh', 'tail_lamp_rh', 'windscreen_front', 'rear_compartment', 'rear_panel', 'rear_quarter_rh', 'door_rear_rh', 'door_rear_lh'
# ]
# targetCol = list(set(targetPart) & set(allTargetCol))
targetCol = allTargetCol


In [7]:
partFPThreshold = {
    # "bumper_front" : 0.3,
    # "headlamp_lh" : 0.3,
    # "headlamp_rh" : 0.3,
    # "bumper_rear":0.3,
    # "misc":0.3,
    # "engine":0.3,
     "cooling_fan" : 0.3,
    "radiator" : 0.3,
    "air_conditioning" : 0.3

}
partFNThreshold = {
    "windscreen_front" : 0.4,
    "windscreen_rear" : 0.4,
    "door_rear_lh" : 0.4,
    "door_rear_rh" : 0.4,
    "door_mirror_lh" : 0.4,
    "door_mirror_rh" : 0.4,
    "door_mirror_rh" : 0.4,
    "rear_quarter_lh" : 0.4,
    "rear_quarter_rh" : 0.4,


}
noClassWeightPart=  [
    "cooling_fan",
    "radiator",
    "air_conditioning"
]

In [8]:

for part in tqdm(targetCol):
    logger.success(f"Start training for part : {part}")
    allCaseIdByPart = []
    allPredByPart = []
    allGtByPart = []
    trainCaseId = trainDf["CaseID"].tolist()
    tempTestDataDf = realTestDataDf
    # tempTestDataDf = realTestDataDf.groupby(part).head(5000)
    testCaseId = tempTestDataDf["CaseID"].tolist()
    # sampledDf = trainDf[allInputFeature + [part]].groupby(part).sample(frac=0.3)
    X_train = trainDf[allInputFeature]
    Y_train = trainDf[part].to_frame()
   
    X_test = tempTestDataDf[allInputFeature]
    Y_test = tempTestDataDf[part].to_frame()

    pos_count = len(Y_test[Y_test[part] == 1]) / len(Y_test)
    neg_count = len(Y_test[Y_test[part] == 0]) / len(Y_test)
    pos_weight = neg_count / pos_count
    train_pool = Pool(
        X_train, Y_train, cat_features=caseFeatures + allVisionFeatures
    )
    test_pool = Pool(X_test, Y_test, cat_features=caseFeatures + allVisionFeatures)
    clsWeight=class_weight.compute_class_weight(class_weight='balanced',classes=np.unique(Y_train.values),y=Y_train.values.squeeze(1))
    logger.success(f"train_class_weight : {clsWeight}")

    testClsWeights = {
        0 : clsWeight[0],
        1 : clsWeight[1]

    }
    print(testClsWeights)
    sampleWeights = class_weight.compute_sample_weight(class_weight=testClsWeights, y=Y_test.values)
    print(f"Shape :{sampleWeights.shape}")
    # prCurve = MulticlassPrecisionRecallCurve(num_classes=2, thresholds=11)
    clf = CatBoostClassifier(
            loss_function="Logloss",
            eval_metric="F1",
            iterations=300,
            early_stopping_rounds=100,
            task_type="GPU",
            class_weights= clsWeight,
            # auto_class_weights="SqrtBalanced",
            use_best_model=True,
            # scale_pos_weight=20
            # class_names=["not_dmg", "dmg"],
        )
    clf.fit(train_pool, eval_set=test_pool, metric_period=50, plot=False, verbose=1)
    test_predict = clf.predict(test_pool)
    # if (part in partFPThreshold):
    #     logger.success(f"Trade off false positive : {part}")
    #     (fpr, tpr, thresholds) = get_roc_curve(clf, test_pool, plot=False)
    #     boundary = select_threshold(clf, curve=(fpr, tpr, thresholds), FPR=partFPThreshold[part])
    #     clf.set_probability_threshold(boundary)
    #     test_predict = clf.predict(X_test)
    # elif part in partFNThreshold:
    #     logger.success(f"Trade off false negative : {part}")
    #     (fpr, tpr, thresholds) = get_roc_curve(clf, test_pool, plot=False)
    #     boundary = select_threshold(clf, curve=(fpr, tpr, thresholds), FNR=partFNThreshold[part])
    #     clf.set_probability_threshold(boundary)
    #     test_predict = clf.predict(X_test)
    acc = accuracy_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)
    confMat = confusion_matrix(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)
    pos_count = len(Y_test[Y_test[part] == 1]) / len(Y_test)
    pos_count_abs = len(Y_test[Y_test[part] == 1])
    # catboostF1 = cat_metrics.F1(Y_test.values.astype(np.int64), test_predict)
    tn = confMat[0][0]
    tp = confMat[1][1]
    fp = confMat[0][1]
    fn = confMat[1][0]
    totalSample = fp + fn + tp + tn
    acc = (tp + tn) / (fp + fn + tp + tn)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 =  (2 * precision * recall) / (precision + recall)
    f1_weighted = f1_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)
    # precision_weighted = precision_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)
    # recall_weighted = recall_score(Y_test.values.astype(np.int64), test_predict, sample_weight=sampleWeights)

    logger.success(f"F1 for part : {part} = {f1}")
    logger.success(f"Weighted F1 scikit for part : {part} = {f1_weighted}")

    # assert (tp / (tp + fn)) + (fn / (tp + fn)) == (tn / (tn + fp)) + (
    #     fp / (tn + fp)
    # )

    allPredModel.append(
        {
            "part": part,
            "tp": tp / (tp + fn),
            "tn": tn / (tn + fp),
            "fp": fp / (tn + fp),
            "fn": fn / (tp + fn),
            "acc": acc,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "pos_count": pos_count,
            "pos_count_abs": pos_count_abs,

        }
    )
    assert len(testCaseId) == len(test_predict)
    assert len(testCaseId) == len(Y_test)

    allPreds.append({"CaseID": testCaseId, part: test_predict})
    allGt.append({"CaseID": testCaseId, part: Y_test.values.squeeze(1)})


  0%|          | 0/34 [00:00<?, ?it/s]2023-01-01 10:06:59.287 | SUCCESS  | __main__:<module>:2 - Start training for part : bonnet
2023-01-01 10:07:00.588 | SUCCESS  | __main__:<module>:25 - train_class_weight : [0.79255996 1.35452569]


{0: 0.7925599589299466, 1: 1.3545256873646967}
Shape :(30000,)
Learning rate set to 0.068371
0:	learn: 1.0000000	test: 0.8244477	best: 0.8244477 (0)	total: 23.4ms	remaining: 7.01s
50:	learn: 1.0000000	test: 0.8244477	best: 0.8244477 (0)	total: 1.56s	remaining: 7.63s


2023-01-01 10:07:04.167 | SUCCESS  | __main__:<module>:80 - F1 for part : bonnet = 0.82444770589294
2023-01-01 10:07:04.167 | SUCCESS  | __main__:<module>:81 - Weighted F1 scikit for part : bonnet = 0.8244477058929194
  3%|▎         | 1/34 [00:04<02:41,  4.88s/it]2023-01-01 10:07:04.168 | SUCCESS  | __main__:<module>:2 - Start training for part : bumper_front


100:	learn: 1.0000000	test: 0.8244477	best: 0.8244477 (0)	total: 3.08s	remaining: 6.06s
bestTest = 0.8244477046
bestIteration = 0
Shrink model to first 1 iterations.


2023-01-01 10:07:05.468 | SUCCESS  | __main__:<module>:25 - train_class_weight : [1.47958727 0.75520952]


...

2023-01-01 10:11:01.284 | SUCCESS  | __main__:<module>:80 - F1 for part : door_rear_lh = 0.6875004287817166
2023-01-01 10:11:01.284 | SUCCESS  | __main__:<module>:81 - Weighted F1 scikit for part : door_rear_lh = 0.6875004287817033
 97%|█████████▋| 33/34 [04:01<00:06,  6.60s/it]2023-01-01 10:11:01.285 | SUCCESS  | __main__:<module>:2 - Start training for part : windscreen_rear


100:	learn: 1.0000000	test: 0.6875004	best: 0.6875004 (0)	total: 2.93s	remaining: 5.78s
bestTest = 0.6875004288
bestIteration = 0
Shrink model to first 1 iterations.


2023-01-01 10:11:02.760 | SUCCESS  | __main__:<module>:25 - train_class_weight : [ 0.52175167 11.99337133]


{0: 0.5217516682248398, 1: 11.993371332066708}
Shape :(30000,)
Learning rate set to 0.068371
0:	learn: 0.9283713	test: 0.6052545	best: 0.6052545 (0)	total: 37.3ms	remaining: 11.2s
50:	learn: 0.9426174	test: 0.6098247	best: 0.6098247 (50)	total: 1.78s	remaining: 8.71s
100:	learn: 0.9454483	test: 0.6107353	best: 0.6111578 (64)	total: 3.48s	remaining: 6.85s
150:	learn: 0.9465907	test: 0.6114748	best: 0.6119547 (139)	total: 5.12s	remaining: 5.06s
200:	learn: 0.9472502	test: 0.6086402	best: 0.6119547 (139)	total: 6.78s	remaining: 3.34s
bestTest = 0.6119547328
bestIteration = 139
Shrink model to first 140 iterations.


2023-01-01 10:11:11.390 | SUCCESS  | __main__:<module>:80 - F1 for part : windscreen_rear = 0.6119547316822763
2023-01-01 10:11:11.390 | SUCCESS  | __main__:<module>:81 - Weighted F1 scikit for part : windscreen_rear = 0.611954731682287
100%|██████████| 34/34 [04:12<00:00,  7.41s/it]


In [9]:

evalMetrics = pd.json_normalize(allPredModel)
avgPrecision = evalMetrics["precision"].mean()
avgRecall = evalMetrics["recall"].mean()
avgF1 = evalMetrics["f1"].mean()
avgTp = evalMetrics["tp"].mean()
avgFp = evalMetrics["fp"].mean()

avgTn = evalMetrics["tn"].mean()
avgAcc = evalMetrics["acc"].mean()
avgFn = evalMetrics["fn"].mean()
avgPosCount = evalMetrics["pos_count"].mean()

print(f"Avg Precision : {avgPrecision}")
print(f"Avg Recall : {avgRecall}")
print(f"Avg F1 : {avgF1}")
print(f"Avg TP : {avgTp}")
print(f"Avg TN : {avgTn}")
print(f"Avg FN : {avgFn}")
print(f"Avg pos_count : {avgPosCount}")

print(f"avgAcc : {avgAcc}")

Avg Precision : 0.7865470398813366
Avg Recall : 0.7088444649071032
Avg F1 : 0.7311291221410756
Avg TP : 0.7088444649071032
Avg TN : 0.8077677604636474
Avg FN : 0.29115553509289677
Avg pos_count : 0.19463823529411767
avgAcc : 0.7617228037760461


In [10]:
fig = px.bar(evalMetrics, x = 'part', y = 'pos_count_abs', text_auto=True)
fig

# fig.update_layout(
#     yaxis = dict(
#         tickmode = 'array',
#         tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
#     )
# )

In [11]:
fig = px.bar(evalMetrics, x = 'part', y = 'pos_count')

fig.add_hline(y=avgPosCount, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
    )
)

In [12]:
fig = px.bar(evalMetrics, x = 'part', y = 'tp')

fig.add_hline(y=avgTp, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTp, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [13]:
fig = px.bar(evalMetrics, x = 'part', y = 'fp')

fig.add_hline(y=avgFp, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgFp, 2), 0.2, 0.5, 0.7],
    )
)

In [14]:
fig = px.bar(evalMetrics, x = 'part', y = 'tn')

fig.add_hline(y=avgTn, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgTn, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [15]:
fig = px.bar(evalMetrics, x = 'part', y = 'fn')

fig.add_hline(y=avgFn, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgFn, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [16]:
fig = px.bar(evalMetrics, x = 'part', y = 'acc')

fig.add_hline(y=avgAcc, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgAcc, 2), 0.2, 0.5, 0.7],
    )
)

In [17]:
fig = px.bar(evalMetrics, x = 'part', y = 'precision')

fig.add_hline(y=avgPrecision, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgPrecision, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [18]:
fig = px.bar(evalMetrics, x = 'part', y = 'recall')

fig.add_hline(y=avgRecall, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgRecall, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [19]:
fig = px.bar(evalMetrics, x = 'part', y = 'f1')

fig.add_hline(y=avgF1, line_width=2, line_color="red")

fig.update_layout(
    yaxis = dict(
        tickmode = 'array',
        tickvals = [np.format_float_positional(avgF1, 2), 0.2, 0.5, 0.7],
    )
)
fig

In [20]:
multilabelPred = pd.DataFrame(pd.DataFrame(allPreds[0]))
for p in tqdm(allPreds[1:]):
    dfPart = pd.DataFrame(p)
    multilabelPred = multilabelPred.merge(dfPart, on="CaseID")
multilabelPred.sort_values(by="CaseID", inplace=True)
multilabelPred

100%|██████████| 33/33 [00:01<00:00, 19.91it/s]


,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,air_conditioning,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
1752,10000038,1,1,1,1,1,1,0,0,1,...,1,1,0,1,1,0,0,0,0,0
1753,10000064,1,1,1,1,1,1,1,1,1,...,1,0,1,0,0,0,1,1,0,0
22560,10000084,0,1,0,0,0,0,0,0,0,...,0,1,0,1,1,1,1,0,0,1
17674,10000104,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17675,10000117,0,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18123,13562091,0,1,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
20419,13562198,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
14074,13562279,0,1,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
18124,13562394,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
multilabelGt = pd.DataFrame(pd.DataFrame(allGt[0]))
for p in tqdm(allGt[1:]):
    dfPart = pd.DataFrame(p)
    multilabelGt = multilabelGt.merge(dfPart, on="CaseID")
multilabelGt.sort_values(by="CaseID", inplace=True)

100%|██████████| 33/33 [00:01<00:00, 18.66it/s]


In [22]:
multilabelGt = multilabelGt[multilabelPred.columns]
multilabelGt

,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,air_conditioning,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
1752,10000038,1,1,1,0,1,1,0,0,1,...,1,1,0,1,1,0,0,0,0,0
1753,10000064,1,1,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22560,10000084,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,0,0
17674,10000104,0,1,1,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17675,10000117,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18123,13562091,0,0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
20419,13562198,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
14074,13562279,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
18124,13562394,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
def hamming_score(y_true, y_pred):
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = np.divide(len(set_true.intersection(set_pred)), 
                    float( len(set_true) ))
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean( [x for x in acc_list if not np.isnan(x)]), acc_list

In [24]:
# tempTargetCol = ["airbag", "undercarriage", "interior"]

In [25]:
subsetAcc2, acc_list = hamming_score(multilabelGt[targetCol].values, multilabelPred[targetCol].values)
subsetAcc2

0.6823338749620634

In [26]:
evalData = pd.DataFrame(
    [
        {
            "metric" : "subset_accuracy",
            "value" : subsetAcc2

        },
         {
            "metric" : "avg_f1",
            "value" : avgF1

        },
         {
            "metric" : "avg_tpr",
            "value" : avgTp

        },
        {
            "metric" : "avg_tnr",
            "value" : avgTn

        },
        {
            "metric" : "part_recall",
            "value" : avgRecall

        },
            {
            "metric" : "part_precision",
            "value" : avgPrecision

        },
                {
            "metric" : "part_accuracy",
            "value" : avgAcc

        }
    ]
)


In [27]:
fig = px.bar(evalData, x = 'metric', y = 'value', text_auto=True)
fig

In [28]:
corr = evalMetrics["tp"].corr(evalMetrics["pos_count"])
corr

0.3954825877082391

In [29]:
corr = evalMetrics["f1"].corr(evalMetrics["pos_count"])
corr

0.20752921747869396

In [30]:
corr = evalMetrics["acc"].corr(evalMetrics["pos_count"])
corr

-0.15615557034871483

In [31]:
targetCol[-1]

'windscreen_rear'

In [32]:
clf.classes_

array([0, 1])

In [33]:
# explainer = shap.Explainer(clf)
# shap_values = explainer(X_test)
# plt.title(targetCol[-1])
# shap.plots.beeswarm(shap_values)


In [34]:
currCaseStudyDf = pd.read_csv("/home/alextay96/Desktop/all_workspace/new_workspace/DLDataPipeline/data/case_study/case_study_2/case_study_gt.csv")
currCaseStudyDf

,Unnamed: 0,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
0,5025,10053736,1,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7519,10142635,1,1,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,19780,10162786,0,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,1,1,0
3,3060,10228504,1,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,23273,10229726,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,11606,10231429,0,1,0,1,1,0,1,0,...,1,0,0,0,0,0,0,0,0,0
6,4379,10284983,1,1,1,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,12628,10293939,0,1,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
8,12685,10301735,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,13748,10353034,0,1,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [35]:
caseStudyPred = multilabelPred[multilabelPred['CaseID'].isin(currCaseStudyDf["CaseID"].unique().tolist())]
caseStudyGt = multilabelGt[multilabelGt["CaseID"].isin(currCaseStudyDf["CaseID"].unique().tolist())]
caseStudyPred.to_csv("case_study_pred.csv")
caseStudyGt.to_csv("case_study_gt.csv")

In [36]:
caseStudyPred

,CaseID,bonnet,bumper_front,grille,fog_lamp_rh,headlamp_lh,headlamp_rh,door_front_lh,door_front_rh,air_conditioning,...,fog_lamp_lh,tail_lamp_rh,windscreen_front,rear_compartment,rear_panel,rear_quarter_rh,door_rear_rh,door_mirror_lh,door_rear_lh,windscreen_rear
5025,10053736,0,1,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7519,10142635,1,1,0,1,1,1,0,1,1,...,1,0,0,0,0,0,0,0,0,0
19780,10162786,0,1,0,0,1,0,1,1,0,...,1,0,0,0,0,0,0,1,1,0
3060,10228504,1,1,0,1,1,1,1,0,1,...,1,0,0,0,0,0,0,1,0,0
23273,10229726,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,1
11606,10231429,0,1,1,1,1,1,1,0,1,...,1,0,0,0,0,0,0,1,0,0
4379,10284983,1,1,1,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
12628,10293939,0,0,0,1,1,1,0,0,0,...,0,0,0,1,1,0,0,0,0,1
12685,10301735,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13748,10353034,0,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
